###### Copyright 2023 The Flower Authors.

# Differential Privacy in flwr Part 1


In this tutorial, we will learn the recommended best practices to build an effective differential privacy setting in federated learning using Flower. 

The documentation will provide a step-by-step guide, code examples, and best practices for implementing user-level differential privacy gurantees in federated learning systems.

You may consider to look at the previous tutorials where we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html)), the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/))..


> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.


In this notebook, we will begin with an indepth presentation of Differential Privacy in federated learning.
 
Let's get started!

## What is Differential Privacy (DP)?
#### _A general definition of DP in Machine Learning._

Differential privacy (DP) is a system used for publicly sharing information about datasets while protecting the privacy of individual data points within the dataset. The main goal of DP is to provide a rigorous approach to ensure that any queries or analysis performed on a dataset do not reveal sensitive or identifiable information about any specific individual in the dataset (data point). DP is a mathematical framework that sets a limit on an individual's influence on the outcome of a computation, such as the parameters of a ML model. Differentially private training algorithms add random noise during training to produce a probability distribution over output models, and ensure that this distribution doesn't change too much given a small change to the training data.

 >DP comes with a parameter $(\epsilon)$ that quantifies how much the distribution could change when adding or removing the training examples of any individual user (the smaller the better). In other words, A smaller $\epsilon$ means that it is less likely that an observer can determine whether a specific individual's data was included in a dataset or not. Additionnally DP also comes with a parameter $(\delta)$ quantifies the risk that an adversary can violate the privacy guarantee of $\epsilon$-differential privacy with a certain probability. For most practical purposes, $\delta$ is set to a very small value (e.g., 1 in a million or less) to ensure that privacy guarantees are maintained even in highly unlikely scenarios.

In nutshell, an algorithm is differentially private if an observer seeing its output cannot tell if a particular individual's information was used in the computation. This makes it challenging for any malicious entity or data analyst to pinpoint individual contributions to the aggregate results. Differential privacy is often discussed in the context of identifying individuals whose information may be in a database [[1]](https://link.springer.com/chapter/10.1007/978-3-540-79228-4_1). 

A randomized mechanism M provides $(\epsilon, \delta)$-differential privacy if for any two neighboring databases, $D_1$ and $D_2$, that differ in only a single record, and for all possible outputs $S \in Range(A)$ : 

$P[M(D_1 ∈ A)] ≤ e^\epsilon P[M(D_2 ∈ A)] + \delta$



Differential privacy continues to be an active field of research, with ongoing efforts to improve and develop new algorithms that strike a balance between data utility and privacy protection. As data collection and analysis become increasingly prevalent in our digital world, differential privacy remains a promising approach to ensuring privacy while allowing for meaningful data analysis and insights.


We claimed to present DP in Federated Learning, let's see what Federated Learning is.


### Quick overview of Federated Learning

Federated Learning is a machine learning technique that allows the training of an algorithm across multiple independent devices or nodes, each with its own local dataset. Unlike traditional centralized machine learning approaches, where data from various sources are merged into one central location for training, federated learning enables training on heterogeneous datasets without explicitly sharing raw data among devices. The primary objective of federated learning is to build a common machine learning model collaboratively among the devices while preserving data privacy and security.

>The previous parts of the tutorial (0-4) give you a deep understanding of Federated Learning and how to get started, you can find it [here](https://flower.dev/docs/framework/tutorial/Flower-0-What-is-FL.html).

### Differential Privacy in Federated Learning

The motivation for using Differential Privacy (DP) in Federated Learning arises from the need to protect user privacy and data confidentiality while collectively training machine learning models across distributed devices or nodes. Federated Learning enhances user privacy by minimizing data exposure to the server and other participants. However, it has been found that traditional Federated Learning is vulnerable to gradient leakage attacks, which could potentially reveal sensitive information about individual user data [[2]](https://arxiv.org/abs/2101.04163). 

By incorporating Differential Privacy into Federated Learning, each device applies DP mechanisms that add carefully calibrated noise to the model updates before sharing them with the central server. This noise introduces randomness, making it difficult for an attacker to infer sensitive details about any individual's data from the aggregated updates.

##### Privacy or Utility?
However, while DP enhances privacy and protects against gradient leakage attacks, it also introduces challenges. One major challenge is balancing the trade-off between privacy protection and model accuracy. The addition of noise can impact the learning process and reduce the accuracy of the resulting model. Thus, there is a need to carefully tune the privacy parameters and the number of iterations to optimize the trade-off between privacy and model accuracy. By incorporating mechanisms like the Laplace and Gaussian mechanisms, researchers have shown how to effectively control the impact of DP noise on model convergence and accuracy [[2]](https://arxiv.org/abs/2101.04163).

The combination of Federated Learning and Differential Privacy provides a robust approach to enable collaborative machine learning across distributed devices while preserving user privacy and data confidentiality. By ensuring that data remains locally processed and incorporating DP mechanisms to add noise during model updates, Federated Learning with Differential Privacy enables organizations to build accurate machine learning models on decentralized data sources without compromising individual privacy.


## Differential Privacy algorithm in Federated Learning: DP-FedAvg

The studies of differential privacy techniques in federated learning can be divided into three categories,[[3]](https://www.scirp.org/journal/paperinformation.aspx?paperid=123374#f3):

#### **Central differential privacy**
Federated learning with central differential privacy is the way that a trusted central server adds noise to global parameters to protect local data. The workflow federated learning with central differential privacy is shown bellow.

<div style="max-width:70%; margin-left: auto; margin-right: auto;">
  <img src="_static/tutorial/fl-central-dp-steps.png" alt="Central Differential privacy"/>
</div>

In the particular case of a Central Differential Privacy setting, a trusted server adds noise to aggregate results in order to protect it against any adversarial attacks. In the framework of FL with central differential privacy, a necessary condition for this framework to be able to secure client data is that the central server is trusted, and if the central server is honest but curious, then the local client’s data (update) will be leaked to the central server.

#### **Local differential privacy**
A more secure framework is FL with local differential privacy (LDP), where each user/client adds noise to the parameters uploaded to the central server after local training iteration to secure the local data updates, while preserving the utility of the model.

<div style="max-width:70%; margin-left: auto; margin-right: auto;">
  <img src="_static/tutorial/fl-local-dp-steps.png" alt="Local Differential Privacy"/>
</div>

Centralized and local differential privacy have shortcomings. Central DP requires a trusted central server, and once the central server is malicious, then the data of the federated learning clients will be compromised, and a trusted central server is practically hard to find. 


 In local differential privacy, although the local data of the federated learning clients are secure under local differential privacy, it leads to too much noise in the aggregated results, which leads to poor privacy-utility trade-offs.
 To address these gaps, researchers have proposed a DP model that can guarantee data security while keeping the amount of added noise limited: distributed differential privacy.

#### **Distributed differential privacy**

 In this setting, each FL client only needs to add a small amount of noise to ensure that the aggregation result of the central server satisfies the central differential privacy. Since the amount of noise added by each client is small and cannot guarantee the security of local data, the clients will use the secure aggregation technique (another part of the tutorial will be dedicate to SecAgg), so that the federated server can only get the aggregation result of the intermediate parameters of all federated clients, but cannot get the intermediate parameters of each federated client, thus securing the federated clients’ local data.

 
<div style="max-width:70%; margin-left: auto; margin-right: auto;">
  <img src="_static/tutorial/fl-distributed-dp-steps.png" alt="Distributed Differential Privacy"/>
</div>


## Final remarks

Congratulations, you just learned the basics of Differential Privacy in federated learning!

In the second part of this tutorial, we are going to build a step by step Differential Privacy algorithm in federated learning with Flower.